In [ ]:
import numpy as np
from collections import Counter

class DecisionTreeClassifier:
    
    def __init__(self, max_depth, min_samples=2, impurity = "gini"):
        self.tree = None
        self.max_depth = max_depth
        self.min_samples = min_samples
        if impurity in ["gini", "entropy"]:
            self.impurity = impurity
        else:
            raise ValueError(f"Invalid impurity: {impurity}. Use 'entropy' or 'gini' instead.")
    
    @staticmethod
    def gini(y):
        imp = 1
        total = len(y)
        counts = Counter(y)
        for cat, count in counts.items():
            p = count/total
            imp -= p**2
        
        return imp

    
    @staticmethod
    def entropy(y):
        total = len(y)
        imp = 0
        counts = Counter(y)
        for cat, count in counts.items():
            p = count/total
            logp = p*math.log2(p)
            imp -= logp
        return imp
    
    def information_gain(self, y,split):
        y_np = y.to_numpy()
        total = len(y_np)
        parent = self.gini(y) if self.impurity == "gini" else self.entropy(y)
        if self.impurity == "gini":
            weighted = sum([(len(idx)/total)*self.gini(y_np[idx]) for idx in split])
        else:
            weighted = sum([(len(idx)/total)*self.entropy(y_np[idx]) for idx in split])
        return parent - weighted
    
    
    def best_split(self,X,y):
        
        # What are X and Y here? For the first node, it is the entire dataset; X being the input and y being the target
        # When we check the information gain in the first node, the target y becomes the parent and based on splits in different features, increment in purity is calculated
        # for subsequent nodes; X and y will only contain a subset of the dataset, all the features will be included
        n_samples, n_features = X.shape
        gain = 0
        thres = None # float value; Only in case of numerical feature
        feat = None # we will keep it as an integer, whenever we are using this to refer to a colummn in a DF, we will use iloc method
        split = None # This will be a list of indices. In case of numerical features, split indices will be identified by a threshold value. In case of categorical feature, the split indices will be identified by indices of each category in that feature
        for i in range(n_features):
            x = X.iloc[:, i]
            
            x = x.to_numpy()
            
            if np.issubdtype(x.dtype, np.number):
                idx = np.argsort(x)
                x_sort = x[idx]
                
                for j in range(1,n_samples):
                    if(x_sort[j] == x_sort[j-1]):
                        continue
                        
                    th = (x_sort[j] + x_sort[j-1])/2.0
                    left = np.where(x<=th)[0]
                    right = np.where(x>th)[0]
                    split_temp = [left, right]
                    new_gain = self.information_gain(y, split_temp)
                    if new_gain > gain:
                        gain = new_gain
                        thres = th
                        split = split_temp
                        feat = i
            else:
                classes = np.unique(x)
                split_temp = [np.where(x==clas)[0] for clas in classes]
                new_gain = self.information_gain(y, split_temp)
                if new_gain > gain:
                    gain = new_gain
                    thres = None
                    split = split_temp
                    feat = i
                    
        return feat, thres, split, gain
            
                    
                    
                    
    def fit(self, X, y):
        self.tree = self.build(X,y,depth=0)
        
    def build(self, X, y, depth=0):
        n_samples, n_features = X.shape
        counts = Counter(y)
        pred_class = max(counts, key=counts.get)
        
        node = {
            "pred": pred_class,
            "feature": None,
            "threshold": None,
            "split": None,
            "children": []
        }
        
        feat, thres, split, gain = self.best_split(X,y)
        
        node.update({"feature": feat, "threshold": thres, "split": split})
        
        if gain <= 0:
            return node
        
        if thres is None:
            x = X.iloc[:, feat]
            x = x.to_numpy()
            classes = np.unique(x)
            for clas in classes:
                idx = np.where(x == clas)[0]
                child = self.build(X.iloc[idx], y.iloc[idx], depth+1)
                node["children"].append((clas,child))
        else:
            left = split[0]
            right = split[1]
            child1 = self.build(X.iloc[left], y.iloc[left])
            child2 = self.build(X.iloc[right], y.iloc[right])
            node["children"].append(("<=", child1))
            node["children"].append((">", child2))
            
        return node
    
    def predict(self, X):
        x_np = X.to_numpy()
        out = [self.predict_one(row, self.tree) for row in x_np]
        return out
    
    def predict_one(self, row, node):
        if node["children"] == []:
            return node["pred"]
        if node["threshold"] == None:
            for cat, child in node["children"]:
                if row[node["feature"]] == cat:
                    return self.preidct_one(row, child)
        else:
            if row[node["feature"]] <= node["threshold"]:
                return self.predict_one(row, node["children"][0][1])
            else:
                return self.predict_one(row, node["children"][1][1])